In [193]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pprint
import pandas as pd

"""
    Make sure you have the following environment variables set:
        SPOTIPY_CLIENT_ID
        SPOTIPY_CLIENT_SECRET
        SPOTIPY_REDIRECT_URI
"""

'DELETE ALL PPRINTS WHEN WE ARE DONE!!!!!!!!!!!!!!!!!!'

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
name = "Drake"


# create main list to store artist info
artist_list = []

# create dict for current artist values
current_artist = {}

# create instance with artist search results
results = spotify.search(q='artist:' + name, type='artist', market='US')
items = results['artists']['items']
artist_uri = ""
if len(items) > 0:
    artist = items[0]
    
    # add artist id to dict
    current_artist['artist_id'] = artist['id']

    # add artist name to dict
    current_artist['artist_name'] = artist['name']
    
    # add external url to dict
    current_artist['external_url'] = artist['external_urls']['spotify']
    
    # add 1st genre to dict
    current_artist['genre'] = artist['genres'][0]
    
    # add image url to dict
    current_artist['image_url'] = artist['images'][0]['url']
    
    # add followers to dict
    current_artist['followers'] = artist['followers']['total']
    
    # add popularity to dict
    current_artist['popularity'] = artist['popularity']
    
    # add type to dict
    current_artist['type'] = artist['type']
    
    # add artist uri to dict
    current_artist['artist_uri'] = artist['uri']
    
    # add dict to list
    artist_list.append(current_artist)
    
    artist_uri = artist["uri"]

    
# create main list to store album info
album_list = []

# create dict for current artist values
current_album = {}

# create instance with album search results for artist
results_alb = spotify.artist_albums(artist_id = artist_uri, album_type = 'album', country = 'US')

album = results_alb['items'][0]

# add album id to dict
current_album['album_id'] = album['id']

# add album name to dict
current_album['album_name'] = album['name']

# add external url to dict
current_album['external_url'] = album['external_urls']['spotify']

# add image url to dict
current_album['image_url'] = album['images'][0]['url']

# add release date to dict
current_album['release_date'] = album['release_date']

# add total number of track to dict
current_album['total_tracks'] = album['total_tracks']

# add type to dict
current_album['type'] = album['type']

# add album uri to dict
current_album['album_uri'] = album['uri']

# add artist id to dict
current_album['artist_id'] = artist['id']

# add dict to list
album_list.append(current_album)

album_id = album['id']


# create main list to store track info
track_list = []
# create main list to store track feature info
track_feat_list = []

# create instance with track search results for album
results_track = spotify.album_tracks(album_id = album_id, limit=50, offset=0)

# iterate over each song in the album
for i in range(len(results_track['items'])):
    
    # create dict for current track values
    current_track = {}
    track = results_track['items'][i]

    # add track id to dict
    current_track['track_id'] = track['id']

    # add song name to dict
    current_track['song_name'] = track['name']

    # add external url to dict
    current_track['external_url'] = track['external_urls']['spotify']

    # add song duration in ms to dict
    current_track['duration_ms'] = track['duration_ms']

    # add boolean value for whether or not song is explicit to dict
    current_track['explicit'] = track['explicit']

    # add disc number to dict
    current_track['disc_number'] = track['disc_number']

    # add type to dict
    current_track['type'] = track['type']

    # add song uri to dict
    current_track['song_uri'] = track['uri']

    # add album id to dict
    current_track['album_id'] = album['id']

    # add dict to list
    track_list.append(current_track)

    track_id = results_track['items'][i]['id']
    
    # create instance with track feature results for track
    results_track_feat = spotify.audio_features([track_id])

    # create dict for current track feature values
    current_track_feat = {}
    track_feat = results_track_feat[0]

    # add track id to dict
    current_track_feat['track_id'] = track_feat['id']

    # add danceability value to dict
    current_track_feat['danceability'] = track_feat['danceability']

    # add energy value to dict
    current_track_feat['energy'] = track_feat['energy']

    # add instrumentalness value to dict
    current_track_feat['instrumentalness'] = track_feat['instrumentalness']

    # add liveness value to dict
    current_track_feat['liveness'] = track_feat['liveness']

    # add loudness value to dict
    current_track_feat['loudness'] = track_feat['loudness']

    # add speechiness value to dict
    current_track_feat['speechiness'] = track_feat['speechiness']

    # add tempo value to dict
    current_track_feat['tempo'] = track_feat['tempo']

    # add type to dict
    current_track_feat['type'] = track_feat['type']

    # add valence value to dict
    current_track_feat['valence'] = track_feat['valence']

    # add song uri to dict
    current_track_feat['uri'] = track_feat['uri']

    # add dict to list
    track_feat_list.append(current_track_feat)


In [196]:
results_alb['items']

[{'album_group': 'album',
  'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'name': 'Drake',
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/3cf4iSSKd8ffTncbtKljXw'},
  'href': 'https://api.spotify.com/v1/albums/3cf4iSSKd8ffTncbtKljXw',
  'id': '3cf4iSSKd8ffTncbtKljXw',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2738dc0d801766a5aa6a33cbe37',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e028dc0d801766a5aa6a33cbe37',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048518dc0d801766a5aa6a33cbe37',
    'width': 64}],
  'name': 'Honestly, Nevermind',
  'release_date': '2022-06-17',
  'release_date_precis

In [197]:
artist_df = pd.DataFrame(artist_list)
artist_df

,artist_id,artist_name,external_url,genre,image_url,followers,popularity,type,artist_uri
0,3TVXtAsR1Inumwj472S9r4,Drake,https://open.spotify.com/artist/3TVXtAsR1Inumw...,canadian hip hop,https://i.scdn.co/image/ab6761610000e5eb429338...,66967787,95,artist,spotify:artist:3TVXtAsR1Inumwj472S9r4


In [198]:
artist_df.columns

Index(['artist_id', 'artist_name', 'external_url', 'genre', 'image_url',
       'followers', 'popularity', 'type', 'artist_uri'],
      dtype='object')

In [199]:
album_df = pd.DataFrame(album_list)
album_df

,album_id,album_name,external_url,image_url,release_date,total_tracks,type,album_uri,artist_id
0,3cf4iSSKd8ffTncbtKljXw,"Honestly, Nevermind",https://open.spotify.com/album/3cf4iSSKd8ffTnc...,https://i.scdn.co/image/ab67616d0000b2738dc0d8...,2022-06-17,14,album,spotify:album:3cf4iSSKd8ffTncbtKljXw,3TVXtAsR1Inumwj472S9r4


In [200]:
album_df.columns

Index(['album_id', 'album_name', 'external_url', 'image_url', 'release_date',
       'total_tracks', 'type', 'album_uri', 'artist_id'],
      dtype='object')

In [201]:
track_df = pd.DataFrame(track_list)
track_df

,track_id,song_name,external_url,duration_ms,explicit,disc_number,type,song_uri,album_id
0,1kNdtonJXHorsVpWerK8C2,Intro,https://open.spotify.com/track/1kNdtonJXHorsVp...,36935,False,1,track,spotify:track:1kNdtonJXHorsVpWerK8C2,3cf4iSSKd8ffTncbtKljXw
1,1vbn9fEyw1IYhqgZJdu9ZB,Falling Back,https://open.spotify.com/track/1vbn9fEyw1IYhqg...,266178,False,1,track,spotify:track:1vbn9fEyw1IYhqgZJdu9ZB,3cf4iSSKd8ffTncbtKljXw
2,4FIgHqXd4KkOwxrGeLDYM2,Texts Go Green,https://open.spotify.com/track/4FIgHqXd4KkOwxr...,308820,False,1,track,spotify:track:4FIgHqXd4KkOwxrGeLDYM2,3cf4iSSKd8ffTncbtKljXw
3,1eCFz60zd7mAXgWLapPd9B,Currents,https://open.spotify.com/track/1eCFz60zd7mAXgW...,157390,False,1,track,spotify:track:1eCFz60zd7mAXgWLapPd9B,3cf4iSSKd8ffTncbtKljXw
4,0nAZGkBGKQCXyaoSJfRhC1,A Keeper,https://open.spotify.com/track/0nAZGkBGKQCXyao...,173097,True,1,track,spotify:track:0nAZGkBGKQCXyaoSJfRhC1,3cf4iSSKd8ffTncbtKljXw
5,7sT7kZEYd1MrmzLLIRVZas,Calling My Name,https://open.spotify.com/track/7sT7kZEYd1MrmzL...,129964,True,1,track,spotify:track:7sT7kZEYd1MrmzLLIRVZas,3cf4iSSKd8ffTncbtKljXw
6,4rmVZajAF7PkrCagGPHbqa,Sticky,https://open.spotify.com/track/4rmVZajAF7PkrCa...,243227,True,1,track,spotify:track:4rmVZajAF7PkrCagGPHbqa,3cf4iSSKd8ffTncbtKljXw
7,28JBD8p18xNuOfyV7Cotdn,Massive,https://open.spotify.com/track/28JBD8p18xNuOfy...,336924,False,1,track,spotify:track:28JBD8p18xNuOfyV7Cotdn,3cf4iSSKd8ffTncbtKljXw
8,6Yj7Zhxt73uvwFFvzQXdxO,Flight's Booked,https://open.spotify.com/track/6Yj7Zhxt73uvwFF...,254407,False,1,track,spotify:track:6Yj7Zhxt73uvwFFvzQXdxO,3cf4iSSKd8ffTncbtKljXw
9,7hRlTK3mgFTBEjX7DjqBsI,Overdrive,https://open.spotify.com/track/7hRlTK3mgFTBEjX...,202451,False,1,track,spotify:track:7hRlTK3mgFTBEjX7DjqBsI,3cf4iSSKd8ffTncbtKljXw


In [202]:
track_df.columns

Index(['track_id', 'song_name', 'external_url', 'duration_ms', 'explicit',
       'disc_number', 'type', 'song_uri', 'album_id'],
      dtype='object')

In [203]:
track_feat_df = pd.DataFrame(track_feat_list)
track_feat_df

,track_id,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,type,valence,uri
0,1kNdtonJXHorsVpWerK8C2,0.193,0.0218,0.880000,0.1170,-31.160,0.0389,125.501,audio_features,0.2130,spotify:track:1kNdtonJXHorsVpWerK8C2
1,1vbn9fEyw1IYhqgZJdu9ZB,0.718,0.7580,0.006530,0.1100,-8.290,0.0703,118.989,audio_features,0.3490,spotify:track:1vbn9fEyw1IYhqgZJdu9ZB
2,4FIgHqXd4KkOwxrGeLDYM2,0.765,0.6270,0.195000,0.0847,-4.607,0.0622,118.997,audio_features,0.5360,spotify:track:4FIgHqXd4KkOwxrGeLDYM2
3,1eCFz60zd7mAXgWLapPd9B,0.883,0.2830,0.001670,0.0935,-12.653,0.1130,122.013,audio_features,0.4160,spotify:track:1eCFz60zd7mAXgWLapPd9B
4,0nAZGkBGKQCXyaoSJfRhC1,0.600,0.4820,0.000104,0.1150,-11.596,0.0701,121.909,audio_features,0.3230,spotify:track:0nAZGkBGKQCXyaoSJfRhC1
5,7sT7kZEYd1MrmzLLIRVZas,0.826,0.2760,0.000085,0.1180,-11.227,0.2660,123.985,audio_features,0.4840,spotify:track:7sT7kZEYd1MrmzLLIRVZas
6,4rmVZajAF7PkrCagGPHbqa,0.853,0.4950,0.000217,0.0844,-14.630,0.1400,137.027,audio_features,0.0774,spotify:track:4rmVZajAF7PkrCagGPHbqa
7,28JBD8p18xNuOfyV7Cotdn,0.499,0.6710,0.016900,0.1480,-6.774,0.0561,124.994,audio_features,0.0557,spotify:track:28JBD8p18xNuOfyV7Cotdn
8,6Yj7Zhxt73uvwFFvzQXdxO,0.593,0.4100,0.099300,0.1150,-13.097,0.0531,122.584,audio_features,0.2010,spotify:track:6Yj7Zhxt73uvwFFvzQXdxO
9,7hRlTK3mgFTBEjX7DjqBsI,0.502,0.6450,0.009640,0.1070,-10.611,0.1220,109.739,audio_features,0.5180,spotify:track:7hRlTK3mgFTBEjX7DjqBsI


In [204]:
track_feat_df.columns

Index(['track_id', 'danceability', 'energy', 'instrumentalness', 'liveness',
       'loudness', 'speechiness', 'tempo', 'type', 'valence', 'uri'],
      dtype='object')